In [ ]:
%%capture
!pip install datasets
!pip install transformers
!pip install librosa
!pip install jiwer
!pip install evaluate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
from datasets import load_dataset

# Load the Common Voice Telugu dataset
from datasets import load_dataset

dataset = load_dataset("openslr", "SLR66", trust_remote_code=True)

# Set a seed for reproducibility
seed = 42

# Split into 80% training and 20% testing
train_test_split = dataset["train"].train_test_split(test_size=0.2, seed=seed)

# Combine the train and validation into one
slr_train = train_test_split['train']  # 80% training
slr_test = train_test_split['test']    # 20% testing

# Check sizes
print(f"Training set size: {len(slr_train)}")
print(slr_train)
print(f"Test set size: {len(slr_test)}")
print(slr_test)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/42.9k [00:00<?, ?B/s]

openslr.py:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4448 [00:00<?, ? examples/s]

Training set size: 3558
Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 3558
})
Test set size: 890
Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 890
})


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(slr_train.remove_columns(["path", "audio"]))

,sentence
0,సియాబోర్గియం ఒక రసాయన మూలకం ఉంది
1,మానవులు ఎక్కువగా కపిలగోవులను పూజిస్తారు
2,మూడు వందల అడుగుల్లో
3,భ్రాజకం అంటేనే ప్రకాశం
4,వానర సైన్యంలో శ్రేష్ఠుడు ఆంజనేయస్వామి
5,చివరకు ఏడుగురిని ఎన్నిక చేసుకొన్నాడు\n
6,ఆపాదించబడిన హోదా అన్ని సమాజాల్లో ఉన్నాయి
7,చిత్తూరు కడప రహదారిపై ఈ గ్రామమున్నది
8,ఇవి తప్ప ఆమె సాహిత్యం నేడు అలభ్యం
9,రెండు వందల అడుగుల్లో


In [ ]:
telugu_special_unwanted_characters = [
    'ఁ',  # Chandrabindu
    'ౄ',  # Vocalic RR
    'ౢ',  # Vocalic L
    'ౣ',  # Vocalic LL
    'ౠ',  # Long Vocalic RR
    'ఽ',  # Avagraha
    '౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯',  # Telugu digits
    'ఀ',  # Telugu Sign Combining Candrabindu Above
    'ౘ',  # Letter TTHA
    'ౙ',  # Letter DDA
    'ౚ',  # Letter RHA
    '౷',  # Vedic Tone
    '‘', '’', '“', '”', '%', '.', ';', '-', ',', '/', '\\', '_', '&',  # Common punctuation
    'G', 'P', 'S', 'e', 'l', 'n', 'r', 't', '\u200c', '\n' #Unwanted in the dataset
]

In [ ]:
import re
chars_to_remove_regex = f'[{re.escape("".join(telugu_special_unwanted_characters))}]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"])
    return batch

In [ ]:
slr_train = slr_train.map(remove_special_characters)
slr_test = slr_test.map(remove_special_characters)

Map:   0%|          | 0/3558 [00:00<?, ? examples/s]

Map:   0%|          | 0/890 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(slr_train.remove_columns(["path", "audio"]))

,sentence
0,అదే అతడి సార్ధక నామధేయం అయింది
1,వరి మరియు ప్రత్తి ఇక్కడ పండించే ప్రధాన పంటలు
2,కేశవరావు తీవ్ర అభ్యంతరం వ్యక్తం చేశాడు
3,ఆమె నిరాకరించి పారిపోయింది
4,అలాంటి సందర్బములొ చేప్పేది ఈ జాతీయము
5,ప్రయాణీకుడు వెనుక సీటులో కూర్చుంటాడు
6,విశాఖపట్నం ఆంధ్ర ప్రదేశ్ రాష్ట్రములోని విశాఖపట్నం జిల్లాకు చెందిన ఒక మండలము
7,చాలా విగ్రహాలు చోరికి గురయ్యాయి
8,లక్షణము శాస్త్రము
9,ఇక అయోమయ నివృత్తి పేజీ తయారు చెయ్యడం సులభం


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = slr_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=slr_train.column_names)
vocab_test = slr_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=slr_test.column_names)

Map:   0%|          | 0/3558 [00:00<?, ? examples/s]

Map:   0%|          | 0/890 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'ం': 1,
 'ః': 2,
 'అ': 3,
 'ఆ': 4,
 'ఇ': 5,
 'ఈ': 6,
 'ఉ': 7,
 'ఊ': 8,
 'ఋ': 9,
 'ఎ': 10,
 'ఏ': 11,
 'ఐ': 12,
 'ఒ': 13,
 'ఓ': 14,
 'ఔ': 15,
 'క': 16,
 'ఖ': 17,
 'గ': 18,
 'ఘ': 19,
 'ఙ': 20,
 'చ': 21,
 'ఛ': 22,
 'జ': 23,
 'ఞ': 24,
 'ట': 25,
 'ఠ': 26,
 'డ': 27,
 'ఢ': 28,
 'ణ': 29,
 'త': 30,
 'థ': 31,
 'ద': 32,
 'ధ': 33,
 'న': 34,
 'ప': 35,
 'ఫ': 36,
 'బ': 37,
 'భ': 38,
 'మ': 39,
 'య': 40,
 'ర': 41,
 'ఱ': 42,
 'ల': 43,
 'ళ': 44,
 'వ': 45,
 'శ': 46,
 'ష': 47,
 'స': 48,
 'హ': 49,
 'ా': 50,
 'ి': 51,
 'ీ': 52,
 'ు': 53,
 'ూ': 54,
 'ృ': 55,
 'ె': 56,
 'ే': 57,
 'ై': 58,
 'ొ': 59,
 'ో': 60,
 'ౌ': 61,
 '్': 62}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

65

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|", clean_up_tokenization_spaces=False)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
repo_name = "wav2vec2-large-xls-r-53-telugu-final-2"

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/kaarthu2003/wav2vec2-large-xls-r-53-telugu-final-2/commit/d82ea17c327e5a84582dffad34ce2e9b1dacf415', commit_message='Upload tokenizer', commit_description='', oid='d82ea17c327e5a84582dffad34ce2e9b1dacf415', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
slr_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/f20a13dc2ab5963431c369a4b088f5c096f6c7230a90b08359007370aaeb6145/tef_04830_00965131543.wav'

In [ ]:
slr_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f20a13dc2ab5963431c369a4b088f5c096f6c7230a90b08359007370aaeb6145/tef_04830_00965131543.wav',
 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00158691,
        -0.00112915, -0.00115967]),
 'sampling_rate': 48000}

In [ ]:
from datasets import Audio
slr_train = slr_train.cast_column("audio", Audio(sampling_rate=16_000))
slr_test = slr_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
slr_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f20a13dc2ab5963431c369a4b088f5c096f6c7230a90b08359007370aaeb6145/tef_04830_00965131543.wav',
 'array': array([ 6.55302210e-06, -4.48762003e-05, -8.45453615e-05, ...,
        -2.42830161e-03, -1.52032159e-03, -1.23449310e-03]),
 'sampling_rate': 16000}

In [ ]:
rand_int = random.randint(0, len(slr_train))

print("Target text:", slr_train[rand_int]["sentence"])
print("Input array shape:", slr_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", slr_train[rand_int]["audio"]["sampling_rate"])

Target text: అతని కుమార్తెను దొంగలు అపహరిస్తారు
Input array shape: (101035,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    batch["labels"] = processor(text=batch["sentence"]).input_ids

    return batch

In [ ]:
slr_train = slr_train.map(prepare_dataset, remove_columns=slr_train.column_names, num_proc = 4)
slr_test = slr_test.map(prepare_dataset, remove_columns=slr_test.column_names, num_proc = 4)

Map (num_proc=4):   0%|          | 0/3558 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/890 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
import evaluate

wer_metric = evaluate.load("wer")

In [ ]:
%%capture
!pip install numpy

In [ ]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    # "facebook/wav2vec2-xls-r-300m",
    'facebook/wav2vec2-large-xlsr-53',
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.05,
    mask_time_prob=0.05,
    layerdrop=0.05,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_encoder()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  eval_strategy="steps",
  num_train_epochs=8,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=400,
  learning_rate=3e-5,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=slr_train,
    eval_dataset=slr_test,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss,Wer
200,No log,17.224897,1.000000
400,14.209600,4.334629,1.000000
600,14.209600,3.920424,1.000000
800,3.589500,3.740389,1.000000
1000,3.589500,3.520171,1.000000
1200,3.464500,3.483521,1.000000
1400,3.464500,3.461531,1.000000
1600,3.431400,3.449619,1.000000


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be rem

TrainOutput(global_step=1776, training_loss=5.900965200888144, metrics={'train_runtime': 3777.7472, 'train_samples_per_second': 7.535, 'train_steps_per_second': 0.47, 'total_flos': 4.0358240789918643e+18, 'train_loss': 5.900965200888144, 'epoch': 7.9820224719101125})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/kaarthu2003/wav2vec2-large-xls-r-53-telugu-final-2/commit/6eae6bfb747076602eb5fdb2eab08e87285c3d1c', commit_message='End of training', commit_description='', oid='6eae6bfb747076602eb5fdb2eab08e87285c3d1c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:55: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: Can't load tokenizer for 'wav2vec2-large-xls-r-53-telugu-final-2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'wav2vec2-large-xls-r-53-telugu-final-2' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))